# ATE end-to-end examples (genriesz)

This notebook demonstrates how to estimate the **Average Treatment Effect (ATE)** with **genriesz**.

We assume the regressor has the form:

- `X = [D, Z...]`, where `D` is a **binary treatment indicator** (`0/1`),
- `Y` is the observed outcome.

We will compute (optionally with cross-fitting):

- **RA**: regression adjustment (plug-in)
- **RW**: Riesz weighting (weighting only)
- **ARW**: augmented Riesz weighting
- **TMLE**: targeted minimum loss estimation (one-step fluctuation)

We also show how to swap the **basis**:
- polynomial features,
- RKHS-style RBF random features,
- nearest-neighbor matching (kNN catchment basis),
- random forest leaf features (optional),
- neural network embeddings (optional).


In [1]:
import numpy as np

from genriesz import (
    grr_ate,
    SquaredGenerator,
    UKLGenerator,
    PolynomialBasis,
    TreatmentInteractionBasis,
    RBFRandomFourierBasis,
    KNNCatchmentBasis,
)

rng = np.random.default_rng(0)


## Synthetic data

In [2]:
# Data-generating process
n = 3000
d_z = 5

Z = rng.normal(size=(n, d_z))

# Treatment assignment: e(Z) = sigmoid(a'Z)
logits = 0.7 * Z[:, 0] - 0.3 * Z[:, 1]
e = 1.0 / (1.0 + np.exp(-logits))
D = rng.binomial(1, e, size=n).astype(float)

# Potential outcomes (constant effect for simplicity)
tau = 1.0
mu0 = 0.5 * Z[:, 0] + 0.25 * Z[:, 1] ** 2
Y0 = mu0 + rng.normal(scale=1.0, size=n)
Y1 = mu0 + tau + rng.normal(scale=1.0, size=n)

Y = D * Y1 + (1.0 - D) * Y0

# Regressor matrix X = [D, Z...]
X = np.column_stack([D, Z])

print("X shape:", X.shape, "Y shape:", Y.shape)


X shape: (3000, 6) Y shape: (3000,)


## Example 1: Polynomial basis + treatment interactions

In [3]:
# Basis on Z, then interact with D (ATE-friendly)
psi = PolynomialBasis(degree=2, include_bias=True)
phi = TreatmentInteractionBasis(base_basis=psi)

# Generator: Squared loss (always safe / no domain constraints)
gen = SquaredGenerator(C=0.0).as_generator()

res_poly = grr_ate(
    X=X,
    Y=Y,
    basis=phi,
    generator=gen,
    cross_fit=True,
    folds=5,
    random_state=0,
    estimators=("ra", "rw", "arw", "tmle"),
    outcome_models="shared",
    riesz_penalty="l2",
    riesz_lam=1e-3,
    max_iter=300,
    tol=1e-8,
)

print(res_poly.summary_text())


ATE estimates (n=3000)
alpha=0.05 | null=0.0
diagnostics: max_abs_smd_unweighted=0.7095986621457625, max_abs_smd_weighted=0.016620078977766132, ess_treated=1188.4096325793273, ess_control=1260.0346792242683

Estimator         Estimate            SE                           CI     p-value
---------------------------------------------------------------------------------
RA                 1.03046    0.00359346         [ 1.02342,  1.03751]           0
RW                 1.03302     0.0569279         [ 0.921445,  1.1446]           0
ARW                1.02916     0.0419132        [ 0.947017,  1.11131]           0
TMLE               1.02921     0.0419128        [ 0.947063,  1.11136]           0


## Example 2: RKHS-style basis (RBF random Fourier features)

This approximates an RBF kernel feature map using random Fourier features, then
interacts the features with treatment.


In [4]:
psi_rff = RBFRandomFourierBasis(
    n_features=500,
    sigma=1.0,
    standardize=True,
    random_state=0,
)
phi_rff = TreatmentInteractionBasis(base_basis=psi_rff)

res_rff = grr_ate(
    X=X,
    Y=Y,
    basis=phi_rff,
    generator=gen,
    cross_fit=True,
    folds=5,
    random_state=0,
    estimators=("ra", "rw", "arw", "tmle"),
    outcome_models="shared",
    riesz_penalty="l2",
    riesz_lam=1e-3,
    max_iter=300,
    tol=1e-8,
)

print(res_rff.summary_text())


ATE estimates (n=3000)
alpha=0.05 | null=0.0
diagnostics: max_abs_smd_unweighted=0.7095986621457625, max_abs_smd_weighted=0.27862992844919376, ess_treated=1363.470230442599, ess_control=1370.9575124421833

Estimator         Estimate            SE                           CI     p-value
---------------------------------------------------------------------------------
RA                 1.15729    0.00717955         [ 1.14321,  1.17136]           0
RW                 1.22541     0.0540494         [ 1.11947,  1.33134]           0
ARW                1.12627     0.0423592         [ 1.04325,  1.20929]           0
TMLE               1.12732     0.0423287         [ 1.04436,  1.21028]           0


## Example 3: Nearest-neighbor matching (kNN catchment-area basis)

Nearest-neighbor matching can be expressed using a **catchment-area indicator basis**

\[
\phi_j(z) = \mathbf{1}\{c_j \in \mathrm{NN}_k(z)\},
\]

and is shown in the paper to be a special case of squared-loss Riesz regression.

Below we compute a matching-style ATE estimate using the catchment basis directly.
For a fully general GRR workflow, you can also pass the catchment basis as `basis=...`
to `grr_ate`.


In [5]:
# Matching-style estimate with a kNN catchment basis
# (This mirrors examples/ate_synthetic_nn_matching.py.)

Z_only = X[:, 1:]  # drop D
n_centers = 400

centers = Z_only[:n_centers]
queries = Z_only[n_centers:]

basis_knn = KNNCatchmentBasis(n_neighbors=1).fit(centers)

# For each query point, w_j counts how often center j is selected
Phi = basis_knn(queries)  # (n_queries, n_centers), dense 0/1
w = Phi.sum(axis=0)       # (n_centers,)

# Matching estimator for ATE:
#   theta_hat = mean_{treated} Y - sum_{controls} w_i Y_i / n_treated
D_cent = D[:n_centers]
Y_cent = Y[:n_centers]

treated = (D_cent == 1)
control = (D_cent == 0)

n_treated = treated.sum()
ate_match = float(Y_cent[treated].mean() - (w[control] @ Y_cent[control]) / n_treated)

print("Matching-style ATE estimate:", ate_match)


Matching-style ATE estimate: 0.24697367793676572


## Example 4: Random forest leaf basis (optional)

If you have `scikit-learn` installed, you can use a random forest as a **feature map**
via leaf indicators. This keeps GRR convex (linear in parameters) while giving a
flexible nonparametric basis.


In [6]:
from sklearn.ensemble import RandomForestRegressor
from genriesz.sklearn_basis import RandomForestLeafBasis

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=6,
    random_state=0,
)

leaf_basis = RandomForestLeafBasis(rf, include_bias=True).fit(X, Y)

res_rf = grr_ate(
    X=X,
    Y=Y,
    basis=leaf_basis,
    generator=gen,
    cross_fit=True,
    folds=5,
    random_state=0,
    estimators=("ra", "rw", "arw", "tmle"),
    outcome_models="shared",
    riesz_penalty="l2",
    riesz_lam=1e-3,
    max_iter=300,
    tol=1e-8,
)

print(res_rf.summary_text())

ATE estimates (n=3000)
alpha=0.05 | null=0.0
diagnostics: max_abs_smd_unweighted=0.7095986621457625, max_abs_smd_weighted=0.786895849078183, ess_treated=350.8795988622065, ess_control=507.56068177539834

Estimator         Estimate            SE                           CI     p-value
---------------------------------------------------------------------------------
RA                 1.03187     0.0199079        [ 0.992854,  1.07089]           0
RW                 1.69371      0.252221         [ 1.19937,  2.18806]    1.88e-11
ARW                1.25382      0.215261        [ 0.831919,  1.67573]    5.72e-09
TMLE                 1.063      0.217126         [ 0.63744,  1.48856]    9.79e-07


## Example 5: Neural network embedding basis (optional)

If you have PyTorch installed, you can use a neural network as a **fixed feature map**.
A recommended workflow is:

1. train an embedding network on a separate task,
2. freeze it,
3. use its outputs as features in GRR.

Below we show the mechanics with a small MLP (training is optional).


In [10]:
import torch
from genriesz.torch_basis import MLPEmbeddingNet, TorchEmbeddingBasis

torch.manual_seed(0)

net = MLPEmbeddingNet(input_dim=X.shape[1], hidden_dims=(64,), output_dim=32)
# (Optional) Train net here on a separate task.
# For a lightweight demo, we skip training and just use the random initialization.
nn_basis = TorchEmbeddingBasis(net, include_bias=True, device="cpu")

res_nn = grr_ate(
    X=X,
    Y=Y,
    basis=nn_basis,
    generator=gen,
    cross_fit=True,
    folds=5,
    random_state=0,
    estimators=("ra", "rw", "arw", "tmle"),
    outcome_models="shared",
    riesz_penalty="l2",
    riesz_lam=1e-3,
    max_iter=300,
    tol=1e-8,
)

print(res_nn.summary_text())

ATE estimates (n=3000)
alpha=0.05 | null=0.0
diagnostics: max_abs_smd_unweighted=0.7095986621457625, max_abs_smd_weighted=0.10074303607722811, ess_treated=1234.797658219504, ess_control=1289.906386252168

Estimator         Estimate            SE                           CI     p-value
---------------------------------------------------------------------------------
RA                  1.0526    0.00233879         [ 1.04801,  1.05718]           0
RW                 1.02999     0.0497571        [ 0.932472,  1.12752]           0
ARW                1.02254     0.0372324        [ 0.949568,  1.09552]           0
TMLE               1.02144     0.0372205        [ 0.948489,  1.09439]           0


## Generator / regularization sweep (SQ / UKL / BP)

This section runs **SQ-Riesz**, **UKL-Riesz**, and **BP-Riesz** for the same
polynomial interaction basis, and compares multiple regularization norms and
strengths.

We use a **branch function** for UKL/BP that forces:

- positive branch for treated units (`D=1`),
- negative branch for control units (`D=0`).

All four estimators (**RA / RW / ARW / TMLE**) are reported.

In [13]:
from genriesz import BPGenerator

branch = lambda x: int(x[0] == 1.0)

generator_grid = [
    ("SQ", SquaredGenerator(C=0.0).as_generator()),
    ("UKL (C=1)", UKLGenerator(C=1.0, branch_fn=branch).as_generator()),
    ("BP (omega=0.1, C=1)", BPGenerator(C=1.0, omega=0.1, branch_fn=branch).as_generator()),
    ("BP (omega=0.2, C=1)", BPGenerator(C=1.0, omega=0.2, branch_fn=branch).as_generator()),
    ("BP (omega=0.5, C=1)", BPGenerator(C=1.0, omega=0.5, branch_fn=branch).as_generator()),
]

penalty_grid = [
    {"penalty": "l2", "lam": 1e-4, "p_norm": None},
    {"penalty": "l2", "lam": 1e-3, "p_norm": None},
    {"penalty": "l1", "lam": 1e-4, "p_norm": None},
    {"penalty": "l1", "lam": 1e-3, "p_norm": 1.5},
]

rows = []
for gname, gen_i in generator_grid:
    for cfg in penalty_grid:
        res_i = grr_ate(
            X=X,
            Y=Y,
            basis=phi,
            generator=gen_i,
            cross_fit=True,
            folds=3,
            random_state=0,
            estimators=("ra", "rw", "arw", "tmle"),
            outcome_models="shared",
            outcome_link="identity",
            riesz_penalty=cfg["penalty"],
            riesz_lam=cfg["lam"],
            riesz_p_norm=cfg["p_norm"],
            max_iter=250,
            tol=1e-8,
        )

        row = {
            "generator": gname,
            "penalty": cfg["penalty"],
            "lam": cfg["lam"],
            "p_norm": cfg["p_norm"],
        }
        for k in ("ra", "rw", "arw", "tmle"):
            e = res_i.estimates[k]
            row[f"{k}"] = e.estimate
            row[f"{k}_se"] = e.se
            row[f"{k}_err"] = e.estimate - tau  # tau is the true constant effect in this DGP
        rows.append(row)

import pandas as pd

df = pd.DataFrame(rows)
df = df.sort_values(by="arw_err", key=lambda s: np.abs(s))
display(df)

generator penalty     lam  p_norm        ra     ra_se    ra_err  \
7             UKL (C=1)      l1  0.0010     1.5  1.027967  0.003809  0.027967   
6             UKL (C=1)      l1  0.0001     NaN  1.027967  0.003809  0.027967   
5             UKL (C=1)      l2  0.0010     NaN  1.027967  0.003809  0.027967   
4             UKL (C=1)      l2  0.0001     NaN  1.027967  0.003809  0.027967   
3                    SQ      l1  0.0010     1.5  1.027967  0.003809  0.027967   
2                    SQ      l1  0.0001     NaN  1.027967  0.003809  0.027967   
0                    SQ      l2  0.0001     NaN  1.027967  0.003809  0.027967   
1                    SQ      l2  0.0010     NaN  1.027967  0.003809  0.027967   
19  BP (omega=0.5, C=1)      l1  0.0010     1.5  1.027967  0.003809  0.027967   
18  BP (omega=0.5, C=1)      l1  0.0001     NaN  1.027967  0.003809  0.027967   
16  BP (omega=0.5, C=1)      l2  0.0001     NaN  1.027967  0.003809  0.027967   
17  BP (omega=0.5, C=1)      l2  0.0010     NaN  1.027967  0.003809  0.027967   
15  BP (omega=0.2, C=1)      l1  0.0010     1.5  1.027967  0.003809  0.027967   
14  BP (omega=0.2, C=1)      l1  0.0001     NaN  1.027967  0.003809  0.027967   
13  BP (omega=0.2, C=1)      l2  0.0010     NaN  1.027967  0.003809  0.027967   
12  BP (omega=0.2, C=1)      l2  0.0001     NaN  1.027967  0.003809  0.027967   
11  BP (omega=0.1, C=1)      l1  0.0010     1.5  1.027967  0.003809  0.027967   
10  BP (omega=0.1, C=1)      l1  0.0001     NaN  1.027967  0.003809  0.027967   
9   BP (omega=0.1, C=1)      l2  0.0010     NaN  1.027967  0.003809  0.027967   
8   BP (omega=0.1, C=1)      l2  0.0001     NaN  1.027967  0.003809  0.027967   

          rw     rw_se    rw_err       arw    arw_se   arw_err      tmle  \
7   1.015917  0.060709  0.015917  1.039322  0.044135  0.039322  1.038294   
6   1.014454  0.060770  0.014454  1.039918  0.044285  0.039918  1.038785   
5   1.015266  0.060731  0.015266  1.039954  0.044264  0.039954  1.038824   
4   1.014382  0.060769  0.014382  1.039978  0.044297  0.039978  1.038835   
3   1.033380  0.056470  0.033380  1.041214  0.041673  0.041214  1.040825   
2   1.033103  0.056626  0.033103  1.041513  0.041770  0.041513  1.041085   
0   1.032923  0.056626  0.032923  1.041555  0.041760  0.041555  1.041128   
1   1.031386  0.056477  0.031386  1.041641  0.041583  0.041641  1.041266   
19  1.089968  0.053456  0.089968  1.041943  0.039425  0.041943  1.042154   
18  1.090072  0.053561  0.090072  1.042124  0.039492  0.042124  1.042314   
16  1.090607  0.053540  0.090607  1.042144  0.039460  0.042144  1.042341   
17  1.091467  0.053515  0.091467  1.042147  0.039441  0.042147  1.042348   
15  1.138767  0.051790  0.138767  1.044217  0.038052  0.044217  1.044799   
14  1.138919  0.051792  0.138919  1.044262  0.038053  0.044262  1.044844   
13  1.139009  0.051789  0.139009  1.044264  0.038051  0.044264  1.044847   
12  1.138927  0.051793  0.138927  1.044265  0.038053  0.044265  1.044847   
11  1.129129  0.051869  0.129129  1.045568  0.038104  0.045568  1.046159   
10  1.129327  0.051869  0.129327  1.045614  0.038103  0.045614  1.046205   
9   1.129354  0.051868  0.129354  1.045614  0.038103  0.045614  1.046206   
8   1.129340  0.051870  0.129340  1.045618  0.038104  0.045618  1.046209   

     tmle_se  tmle_err  
7   0.044140  0.038294  
6   0.044288  0.038785  
5   0.044267  0.038824  
4   0.044299  0.038835  
3   0.041676  0.040825  
2   0.041772  0.041085  
0   0.041763  0.041128  
1   0.041585  0.041266  
19  0.039423  0.042154  
18  0.039490  0.042314  
16  0.039458  0.042341  
17  0.039438  0.042348  
15  0.038040  0.044799  
14  0.038041  0.044844  
13  0.038039  0.044847  
12  0.038042  0.044847  
11  0.038083  0.046159  
10  0.038083  0.046205  
9   0.038082  0.046206  
8   0.038083  0.046209